# Building AI-powered apps on Azure SQL Database using LLMs and LangChain

Azure SQL Database now supports native vector search capabilities, bringing the power of vector search operations directly to your SQL databases. You can read the full announcement of the public preview [here](https:\devblogs.microsoft.com\azure-sql\exciting-announcement-public-preview-of-native-vector-support-in-azure-sql-database\)

We are also thrilled to announce the release of [langchain-sqlserver](https:\pypi.org\project\langchain-sqlserver\) version 0.1.1. You can use this package to manage Langchain vectorstores in SQL Server. This new release brings enhanced capabilities by parsing both ODBC connection strings and SQLAlchemy format connection strings, making it easier than ever to integrate with Azure SQL DB

In this step-by-step tutorial, we will show you how to add generative AI features to your own applications with just a few lines of code using Azure SQL DB, [LangChain](https:\pypi.org\project\langchain-sqlserver\), and LLMs.

## Dataset

The Harry Potter series, written by J.K. Rowling, is a globally beloved collection of seven books that follow the journey of a young wizard, Harry Potter, and his friends as they battle the dark forces led by the evil Voldemort. Its captivating plot, rich characters, and imaginative world have made it one of the most famous and cherished series in literary history. 

This Sample dataset from [Kaggle](https:\www.kaggle.com\datasets\shubhammaindola\harry-potter-books) contains 7 .txt files of 7 books of Harry Potter. For this demo we will only be using the first book - Harry Potter and the Sorcerer's Stone.

In this notebook, we will showcase two exciting use cases:
1. A sample Python application that can understand and respond to human language queries about the data stored in your Azure SQL Database. This **Q&A system** leverages the power of SQL Vectore Store & LangChain to provide accurate and context-rich answers from the Harry Potter Book.
1. Next, we will push the creative limits of the application by teaching it to generate new AI-driven **Harry Potter fan fiction** based on our existing dataset of Harry Potter books. This feature is sure to delight Potterheads, allowing them to explore new adventures and create their own magical stories.

## Prerequisites

- **Azure Subscription**: [Create one for free](https:\azure.microsoft.com\free\cognitive-services?azure-portal=true)
    
- **Azure SQL Database**: [Set up your database for free](https:\learn.microsoft.com\azure\azure-sql\database\free-offer?view=azuresql)
    
- **Azure OpenAI Access**: Apply for access in the desired Azure subscription at [https://aka.ms/oai/access](https:\aka.ms\oai\access)
    
- **Azure OpenAI Resource**: Deploy an embeddings model (e.g., `text-embedding-small` or `text-embedding-ada-002`) and a `GPT-4.0` model for chat completion. Refer to the [resource deployment guide](https:\learn.microsoft.com\azure\ai-services\openai\how-to\create-resource) 

- **Azure Blob Storage** Deploy a Azure [Blob Storage Account](https:\learn.microsoft.com\azure\storage\blobs\storage-quickstart-blobs-portal) to upload your dataset
    
- **Python**: Version 3.7.1 or later from Python.org. (Sample has been tested with Python 3.11)
    
- **Python Libraries**: Install the required libraries from the requirements.txt
    
- **Jupyter Notebooks**: Use within [Azure Data Studio](https:\learn.microsoft.com\en-us\azure-data-studio\notebooks\notebooks-guidance) or Visual Studio Code .
    

## Getting Started

1. **Model Deployment**: Deploy an embeddings model (`text-embedding-small` or `text-embedding-ada-002`) and a `GPT-4` model for chat completion. Note the 2 models deployment names for use in the `.env` file

![Deployed OpenAI Models](..\Assets\modeldeployment.png)

2. **Connection String**: Find your Azure SQL DB connection string in the Azure portal under your database settings.
3. **Configuration**: Populate the `.env` file with your SQL server connection details , Azure OpenAI key and endpoint , api-version & Model deploymentname

You can retrieve the Azure OpenAI _endpoint_ and _key_:

![Azure OpenAI Endpoint and Key](..\Assets\endpoint.png)

4. **Upload dataset** In your [Blob Storage Account](https:\learn.microsoft.com\en-us\azure\storage\blobs\storage-quickstart-blobs-portal) create a container and upload the .txt file using the steps [here](https:\learn.microsoft.com\azure\storage\blobs\storage-quickstart-blobs-portal)

## Running the Notebook

To [execute the notebook](https:\learn.microsoft.com\azure-data-studio\notebooks\notebooks-python-kernel), connect to your Azure SQL database using Azure Data Studio, which can be downloaded [here](https:\azure.microsoft.com\products\data-studio)

In [ ]:
#Setup the python libraries required for this notebook
#Please ensure that you navigate to the directory containing the `requirements.txt` file in your terminal
%pip install -r requirements.txt

In [49]:
#Load the env details
from dotenv import load_dotenv
load_dotenv()

True

## Install the new [langchain-sqlserver](https://pypi.org/project/langchain-sqlserver/) python package.

The code lives in an integration package called:[langchain-sqlserver](https://github.com/langchain-ai/langchain-azure/tree/main/libs/sqlserver).

In [ ]:
!pip install langchain-sqlserver==0.1.1

## Loading Our Harry Potter Dataset

In this example, we will use a dataset consisting of text files from the Harry Potter books, which are stored in Azure Blob Storage. This dataset will be used to demonstrate the capabilities of LangChain for RAG (question-answering and fan fiction generation)

LangChain has a seamless integration with [AzureBlobStorage](https://python.langchain.com/docs/integrations/document_loaders/azure_blob_storage_container/), making it easy to load documents directly from Azure Blob Storage. 

Additionally, LangChain provides a method to [split long text](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) into smaller chunks, using langchain-text-splitter which is essential since Azure OpenAI embeddings have an input token limit.



In [50]:
import magic
from langchain.document_loaders import AzureBlobStorageFileLoader
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables from a .env file
load_dotenv()

# Get the connection string and container name from the environment variables
conn_str = os.getenv("AZURE_CONN_STR")
container_name = os.getenv("AZURE_CONTAINER_NAME")
blob_name = "01 Harry Potter and the Sorcerers Stone.txt" #Name of the .txt file

# Create an instance of AzureBlobStorageFileLoader
loader = AzureBlobStorageFileLoader(conn_str=conn_str, container=container_name, blob_name=blob_name)

# Load the document from Azure Blob Storage
documents = loader.load()

# Split the document into smaller chunks if necessary
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

# Print the number of split documents
print(f"Number of split documents: {len(split_documents)}")


Number of split documents: 572


## Define function for Embedding Generation & Chat Completion


In this example we use Azure OpenAI to generate embeddings of the split documents, however you can use any of the different embeddings provided in LangChain.

In [51]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

#Use environment variables
azure_endpoint = os.getenv("AZURE_ENDPOINT")
azure_deployment_chatcompletion_name = os.getenv("AZURE_DEPLOYMENT_CHATCOMPLETION_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")
azure_api_key = os.getenv("AZURE_API_KEY")
azure_deployment_embedding_name = os.getenv("AZURE_DEPLOYMENT_EMBEDDING_NAME")
connection_string = os.getenv("CONNECTION_STRING")


#Use AzureChatOpenAI for chat completions
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment_chatcompletion_name,
    openai_api_version=azure_api_version,
    openai_api_key=azure_api_key
)

#Use AzureOpenAIEmbeddings for embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_deployment_embedding_name, 
    openai_api_version=azure_api_version,
    openai_api_key=azure_api_key
)

## Initialize the Vector Store & insert the documents into Azure SQL with their embeddings

After splitting the long text files of Harry Potter books into smaller chunks, you can generate vector embeddings for each chunk using the Text Embedding Model available through [AzureOpenAI](https://python.langchain.com/docs/integrations/llms/azure_openai/). Notice how we can accomplish this in just a few lines of code!

- First, initialize the vector store and set up the embeddings using AzureOpenAI
- Once we have our Vector Store we can add items to our vector store by using the add\_documents function.

In [52]:
from langchain_sqlserver import SQLServer_VectorStore
from langchain_community.vectorstores.utils import DistanceStrategy

from dotenv import load_dotenv
load_dotenv()
connection_string = os.getenv("CONNECTION_STRING")


# Initialize the vector store
vector_store = SQLServer_VectorStore(
    connection_string=connection_string,
    distance_strategy=DistanceStrategy.COSINE, #optional, if not provided, defaults to COSINE
    embedding_function=embeddings, # you can use different embeddings provided in LangChain
    embedding_length=1536,
    table_name = "hpbook_1" #using a table with custom name
)  

# Add split documents to the vector store individually
for i, doc in enumerate(split_documents):
    vector_store.add_documents(documents=[doc], ids=[f"doc_{i}"])

print("Documents added to the vector store successfully!")

Documents added to the vector store successfully!


## Querying Data - Similarity Search:

Once your vector store has been created and the relevant documents have been added you will most likely wish to query it during the running of your chain or agent.

The vectorstore also supports a set of filters that can be applied against the metadata fields of the documents. By applying filters based on specific metadata attributes, users can limit the scope of their searches, concentrating only on the most relevant data subsets.

Performing a simple similarity search can be done as follows with the `similarity_search_with_score`

In [53]:
from typing import List, Tuple

# Perform similarity search
query = "Wizarding world snacks"
docs_with_score: List[Tuple[Document, float]] = vector_store.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Source Doc: ", doc.metadata.get("source", "N/A").split('/')[-1] )
    print("Content: ", doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.42822275405758725
Source Doc:  01 Harry Potter and the Sorcerers Stone.txt
Content:  Around half past twelve there was a great clattering outside in the corridor and a smiling, dimpled woman slid back their door and said, “Anything off the cart, dears?”

Harry, who hadn’t had any breakfast, leapt to his feet, but Ron’s ears went pink again and he muttered that he’d brought sandwiches. Harry went out into the corridor.

He had never had any money for candy with the Dursleys, and now that he had pockets rattling with gold and silver he was ready to buy as many Mars Bars as he could carry — but the woman didn’t have Mars Bars. What she did have were Bettie Bott’s Every Flavor Beans, Drooble’s Best Blowing Gum, Chocolate Frogs. Pumpkin Pasties, Cauldron Cakes, Licorice Wands, and a number of other strange things Harry had never seen in his life. Not wanting to miss anything, he got some of everything

# RAG with SQLDB, Langchain & LLMs:

## Use Case 1: Q&A System based on the Story Book

The Q&A function allows users to ask specific questions about the story, characters, and events, and get concise, context-rich answers. This not only enhances their understanding of the books but also makes them feel like they're part of the magical universe.

The LangChain Vector store simplifies building sophisticated Q&A systems by enabling efficient **similarity searches** to find the top 10 relevant documents based on the user's query. The retriever is created from the **vector\_store,** and the question-answer chain is built using the **create\_stuff\_documents\_chain** function. A prompt template is crafted using the **ChatPromptTemplate** class, ensuring structured and context-rich responses. Often in Q&A applications it's important to show users the sources that were used to generate the answer. LangChain's built-in **create\_retrieval\_chain** will propagate retrieved source documents to the output under the "**context**" key:

Read more about Langchain RAG tutorials & the terminologies mentioned above [here](https:\python.langchain.com\docs\tutorials\rag\)

In [54]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Tuple
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Define the function to perform the RAG chain invocation 
def get_answer_and_sources(user_query: str):
    # Perform similarity search with scores
    docs_with_score: List[Tuple[Document, float]] = vector_store.similarity_search_with_score(
        user_query, 
        k=10, 
    )

    # Extract the context from the top results
    context = "\n".join([doc.page_content for doc, score in docs_with_score])

    # Define the system prompt
    system_prompt = (
        "You are an assistant for question-answering tasks based on the story in the book. "
        "Use the following pieces of retrieved context to answer the question. "
        "If you don't know the answer, say that you don't know, but also suggest that the user can use the fan fiction function to generate fun stories. "
        "Use 5 sentences maximum and keep the answer concise by also providing some background context of 1-2 sentences."
        "\n\n"
        "{context}"
    )

    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    # Create the retriever and chains
    retriever = vector_store.as_retriever()
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Define the input
    input_data = {"input": user_query}

    # Invoke the RAG chain
    response = rag_chain.invoke(input_data)

    # Print the answer
    print("Answer:", response["answer"])

    
    # Prepare the data for the table
    data = {
         "Doc ID": [doc.metadata.get("source", "N/A").split('/')[-1] for doc in response["context"]],
        "Content": [doc.page_content[:50] + "..." if len(doc.page_content) > 100 else doc.page_content for doc in response["context"]],
    }


    # Create a DataFrame
    df = pd.DataFrame(data)

    # Print the table
    print("\nSources:")
    print(df.to_markdown(index=False))


In [56]:
# Define the user query
user_query = "How did Harry feel when he first learnt that he was a Wizard?"

# Call the function to get the answer and sources
get_answer_and_sources(user_query)

Answer: When Harry first learned that he was a wizard, he felt quite sure there had been a horrible mistake. He struggled to believe it because he had spent his life being bullied and mistreated by the Dursleys. If he was really a wizard, he wondered why he hadn't been able to use magic to defend himself. This disbelief and surprise were evident when he gasped, “I’m a what?”

Sources:
| Doc ID                                      | Content                                               |
|:--------------------------------------------|:------------------------------------------------------|
| 01 Harry Potter and the Sorcerers Stone.txt | Harry was wondering what a wizard did once he’d fi... |
| 01 Harry Potter and the Sorcerers Stone.txt | Harry realized his mouth was open and closed it qu... |
| 01 Harry Potter and the Sorcerers Stone.txt | “Most of us reckon he’s still out there somewhere ... |
| 01 Harry Potter and the Sorcerers Stone.txt | “Ah, go boil yer heads, both of yeh,” said H

In [57]:
# Define the user query
user_query = "Did Harry have a pet? What was it"

# Call the function to get the answer and sources
get_answer_and_sources(user_query)

Answer: Yes, Harry had a pet owl named Hedwig. He decided to call her Hedwig after finding the name in a book titled *A History of Magic*.

Sources:
| Doc ID                                      | Content                                               |
|:--------------------------------------------|:------------------------------------------------------|
| 01 Harry Potter and the Sorcerers Stone.txt | Harry sank down next to the bowl of peas. “What di... |
| 01 Harry Potter and the Sorcerers Stone.txt | Harry kept to his room, with his new owl for compa... |
| 01 Harry Potter and the Sorcerers Stone.txt | As the snake slid swiftly past him, Harry could ha... |
| 01 Harry Potter and the Sorcerers Stone.txt | Ron reached inside his jacket and pulled out a fat... |


## Use Case 2 : Generate fan fiction based on user prompts:

Potterheads are known for their creativity and passion for the series. With this they can craft their own stories based on user prompt given , explore new adventures, and even create alternate endings. Whether it's imagining a new duel between Harry and Voldemort or crafting a personalized Hogwarts bedtime story for you kiddo, the possibilities are endless.

The fan fiction function uses the embeddings in the vector store to generate new stories :

- **Retrieving Relevant Passages**: When a user provides a prompt for a fan fiction story, the function first retrieves relevant passages from the SQL vector store. The vector store contains embeddings of the text from the Harry Potter books, which allows it to find passages that are contextually similar to the user's prompt.

- **Formatting the Retrieved Passages**: The retrieved passages are then formatted into a coherent context. This involves combining the text from the retrieved passages into a single string that can be used as input for the language model.

- **Generating the Story:** The formatted context, along with the user's prompt, is fed into a language model GPT4o to generate the fan fiction story. The language model uses the context to ensure that the generated story is relevant and coherent, incorporating elements from the retrieved passages.

In [ ]:
# Define the function to perform the RAG chain invocation and create the DataFrame
def generate_fan_fiction(user_query: str):
    # Perform similarity search with scores
    docs_with_score: List[Tuple[Document, float]] = vector_store.similarity_search_with_score(
        user_query, 
        k=10 
    )

    # Extract the context from the top results
    context = "\n".join([doc.page_content for doc, score in docs_with_score])

    # Define the system prompt
    system_prompt = (
        "You are an assistant for generating fan fiction bedtime stories for Harry Potter series fans"
        "Use the following pieces of retrieved context to create a story based on the prompt. "
        "Be creative and engaging."
        "Limit the story to 15 sentences"
        "\n\n"
        "{context}"
    )

    # Create the prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    # Create the retriever and chains
    retriever = vector_store.as_retriever()
    story_generation_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, story_generation_chain)

    # Define the input
    input_data = {"input": user_query}

    # Invoke the RAG chain
    response = rag_chain.invoke(input_data)

    # Print the generated story
    print("Generated Story:", response["answer"])
    print("-" * 80)

    # Prepare the data for the table
    data = {
        "Doc ID": [doc.metadata.get("source", "N/A").split('/')[-1] for doc, score in docs_with_score],
        "Score": [f"{score:.2f}" for doc, score in docs_with_score],
        "Content": [doc.page_content[:50] + "..." if len(doc.page_content) > 100 else doc.page_content for doc, score in docs_with_score],
    }

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Print the table
    print("\nSources for Inspiration:")
    print(df.to_markdown(index=False))

In [ ]:
# Define the user query
user_query = "Write a short story about how Harry meets a boy called Davide Mauri on Hogwarts express on their first day. Davide also tells Harry & the others about Native Vector Support feature Azure SQL DB going Public preview in the Muggle world"

# Call the function to generate the fan fiction story
generate_fan_fiction(user_query)

Thus Combining the Q&A system with the fan fiction generator offers a unique and immersive reading experience. If users come across a puzzling moment in the books, they can ask the Q&A system for clarification. If they're inspired by a particular scene, they can use the fan fiction generator to expand on it and create their own version of events. This interactive approach makes reading more engaging and enjoyable.